# 1. 자연어 처리

In [13]:
!pip install konlpy

from konlpy.tag import Okt
okt=Okt()
def tokenizer(text):  #함수를 만드는 부분
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.nouns(text) #nouns를 morphs로 바꾸면 명사외에도...
    return tokens_ko


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 32.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df = pd.read_csv("news.csv")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vect = CountVectorizer(tokenizer=tokenizer)
dtm = tf_vect.fit_transform( df.articles )
dtm

# 2. 웹수집

In [1]:
from bs4 import BeautifulSoup
import urllib
import numpy as np
import pandas as pd
import time

#----------------------------------------------#
#변경할 부분
startdate = "2025.07.28"
enddate = "2025.08.01"
searchword = "삼성전자" #검색어
#----------------------------------------------#

base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query"+urllib.parse.urlencode([('',searchword)], doseq=True)+"&sort=1&photo=3&field=0&pd=3&ds="+startdate+"&de="+enddate+"&mynews=1&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from"+startdate.replace(".", "")+"to"+enddate.replace(".", "")+",a:all&mynews=0&start="
titles = [ ]
urls = [ ]
i=1

while True:
 try:
    time.sleep(0.45)
    url = base_url+str(i)
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    t_url = soup.select('a[href^="https://n.news.naver.com/mnews/article/"]')  #클래스 표시
    t_len = len(t_url)
    if t_len <1:
        break

    for j in np.arange(t_len)+1:
            try:
                url = soup.select('a[href^="https://n.news.naver.com/mnews/article/"]')[j]['href']
                urls.append(url)
            except Exception as e:
                continue
    i=i+10

 except Exception as e:
    continue

result = pd.DataFrame({"Url": urls})
#result.Url[0] #1행 출력

 - 본문 수집

In [2]:
articles = []
dates = []
likes = []

for i in result.Url:
   try:
    if( "news.naver.com" in i ):
     print(i)
     time.sleep(0.45)
     html = urllib.request.urlopen(i).read()
     soup = BeautifulSoup(html, 'html.parser')
    article = soup.find('article', {'id': 'dic_area'})
    for tag in article.find_all(['span', 'strong']):
      tag.decompose()
    for br in article.find_all('br'):
      br.replace_with('\n')
    news = article.get_text(strip=True, separator='\n')
    articles.append(news)
    time_span = soup.find('span', class_='_ARTICLE_DATE_TIME')
    news_time = time_span.get('data-date-time')
    dates.append(news_time)
   except Exception as e:
    print(e)
    continue


https://n.news.naver.com/mnews/article/005/0001793404?sid=101
https://n.news.naver.com/mnews/article/469/0000879423?sid=104
https://n.news.naver.com/mnews/article/469/0000879414?sid=101
https://n.news.naver.com/mnews/article/009/0005534960?sid=105
https://n.news.naver.com/mnews/article/009/0005534945?sid=101
https://n.news.naver.com/mnews/article/015/0005165838?sid=101
https://n.news.naver.com/mnews/article/009/0005534885?sid=103
https://n.news.naver.com/mnews/article/015/0005165813?sid=101
https://n.news.naver.com/mnews/article/015/0005165812?sid=101
https://n.news.naver.com/mnews/article/015/0005165807?sid=104
https://n.news.naver.com/mnews/article/015/0005165805?sid=101
https://n.news.naver.com/mnews/article/469/0000879391?sid=101
https://n.news.naver.com/mnews/article/021/0002726809?sid=101
https://n.news.naver.com/mnews/article/021/0002726782?sid=101
https://n.news.naver.com/mnews/article/016/0002508457?sid=101
https://n.news.naver.com/mnews/article/016/0002508464?sid=101
https://

In [3]:
# 수집된 내용을 표로 정리하기, news컬럼, date컬럼
train_data = pd.DataFrame({'articles':articles, 'date':dates})
train_data

,articles,date
0,이재명정부 출범 이후 급등세를 이어오던 코스피 지수가 1일 전장보다 약 3.9% 급...,2025-08-01 19:11:14
1,대만이 도널드 트럼프 미국 행정부의 20% 고율 관세 부과에 당혹감을 감추지 못하고...,2025-08-01 19:01:17
2,"상호관세 협상 여파가 가시지 않은 가운데, 세제개편안에 대한 실망감이 더해지며 코스...",2025-08-01 18:30:12
3,삼성전자 최신 폴더블 스마트폰 '갤럭시 Z 폴드7'(사진)이 미국 시장 사전판매에서...,2025-08-01 18:23:45
4,"애플, 마이크로소프트(MS), 메타 등 글로벌 빅테크 업체들이 인공지능(AI) 인프...",2025-08-01 18:23:09
...,...,...
404,“공장 운영에 필요한 인허가를 받으려면 예전엔 자치단체를 몇 번씩이나 찾아가야 했잖...,2025-07-28 03:04:14
405,"직장인 박모 씨(35)는 지난해 말 이사하면서 기존에 쓰던 세탁기, 건조기, 청소기...",2025-07-28 03:03:11
406,LG전자 TV 사업 부문이 올 2분기 적자 성적표를 받으면서 TV 사업에 대한 우려...,2025-07-28 02:08:06
407,정보기술(IT) 기업에 다니는 A씨(33)는 요즘 퇴근 후 자기소개서 작성에 여념이...,2025-07-28 00:31:11


In [4]:
train_data.to_csv("news_2025.csv")

In [12]:
train_data["time"] = pd.to_datetime(train_data["date"]).dt.date
train_data.head()

,articles,date,time
0,이재명정부 출범 이후 급등세를 이어오던 코스피 지수가 1일 전장보다 약 3.9% 급...,2025-08-01 19:11:14,2025-08-01
1,대만이 도널드 트럼프 미국 행정부의 20% 고율 관세 부과에 당혹감을 감추지 못하고...,2025-08-01 19:01:17,2025-08-01
2,"상호관세 협상 여파가 가시지 않은 가운데, 세제개편안에 대한 실망감이 더해지며 코스...",2025-08-01 18:30:12,2025-08-01
3,삼성전자 최신 폴더블 스마트폰 '갤럭시 Z 폴드7'(사진)이 미국 시장 사전판매에서...,2025-08-01 18:23:45,2025-08-01
4,"애플, 마이크로소프트(MS), 메타 등 글로벌 빅테크 업체들이 인공지능(AI) 인프...",2025-08-01 18:23:09,2025-08-01


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vect = CountVectorizer(tokenizer=tokenizer, max_features=100)
dtm = tf_vect.fit_transform( train_data.articles )
dtm

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 15832 stored elements and shape (409, 100)>

In [18]:
df = pd.DataFrame( dtm.toarray(), index=train_data.time)

In [19]:
df.groupby("time").sum()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
time,,,,,,,,,,,,,,,,,,,,,
2025-07-28,46,427,57,32,225,44,122,84,88,120,...,93,57,266,74,92,49,46,105,46,79
2025-07-29,48,310,35,38,114,31,127,42,45,87,...,154,43,124,46,118,53,65,181,53,130
2025-07-30,45,462,42,51,75,59,185,34,44,86,...,222,150,40,24,257,87,102,492,76,168
2025-07-31,36,428,45,89,44,81,231,60,69,61,...,244,93,87,43,227,67,92,347,81,166
2025-08-01,21,157,16,17,11,13,64,28,21,25,...,69,18,28,15,64,15,17,59,47,24
